In [3]:
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np

In [ ]:
# Define global variables.
hidden_size = 100
num_classes = 228
learning_rate = 3e-4
num_train_steps = 100

In [6]:
def model_fn(features, labels, mode):
    """Model function for fashion classes predictions.
    
    Inputs:
        features: dict.
            Required key: "image". The value needs to have shape (batch_size, 299, 299, 3),
                and each value needs to be in range [0, 1].
        labels: shape (batch_size, num_classes)
        mode: tf.estimator.ModeKeys.(PREDICT|TRAIN|EVAL)
    
    Returns:
        estimator_spec.
    """
    # class_prob > threshold will be outputted.
    thresholds =[0.3, 0.5, 0.7]
    threshold = 0.5
#     threshold = tf.get_variable('threshold_unbound', initializer=0.5)
#     threshold = tf.clip_by_value(threshold, 0.1, 0.9, name='threshold')
    
    # Input layer.
    images = features['image']
    module = hub.Module("https://tfhub.dev/google/imagenet/inception_resnet_v2/feature_vector/1")
    features = module(images)  # (batch_size, D)
    
    # Create multi-head sigmoid outputs.
    # It measures the independent probability of a class showing in the image.
    raw_logits = tf.contrib.layers.fully_connected(
        inputs=features,
        num_outputs=num_classes,
        activation_fn=None)  # (batch_size, num_classes)
    
    raw_probs = tf.sigmoid(raw_logits)  # (batch_size, num_classes)
    
#     # RNN layer.
#     gru_cell = tf.nn.rnn_cell.GRUCell(hidden_size)
#     outputs, _ = tf.nn.dynamic_rnn(cell=gru_cell, inputs=)
    
    predictions = {
        'classes': raw_probs > threshold,
        'probs': raw_probs,
    }
    
    # PREDICT mode.
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)
    
    # Calculate loss (for both TRAIN and EVAL mode).
    loss = tf.losses.sigmoid_cross_entropy(
        multi_class_labels=labels,
        logits=raw_logits)
    
    # TRAIN mode.
    if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
        train_op = optimizer.minimize(
            loss=loss,
            global_step=tf.train.get_global_step())
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)    
    
    # Add evalutaion metrics (for EVAL mode).
    precisions = tf.metrics.precision_at_thresholds(
        labels=labels,
        predictions=predictions['probs'],
        thresholds=thresholds)
    recalls = tf.metrics.recall_at_thresholds(
        labels=labels,
        predictions=predictions['probs'],
        thresholds=thresholds)
    auc = tf.metrics.auc(
        labels=labels,
        predictions=predictions['probs'],
        thresholds=thresholds)
    
    eval_metric_ops = {
        'precisions': precisions,
        'recalls': recalls,
        'auc': auc,
    }
    return tf.estimator.EstimatorSpec(mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)

In [7]:
# Create the estimator.
classifier = tf.estimator.Estimator(
    model_fn=model_fn,
    model_dir='model_dir/baseline')

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'model_dir/baseline', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f94183ebe80>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [ ]:
# TODO: Define input_fn.
classifier.train(input_fn, steps=num_train_steps)